In [350]:
import xarray as xr
import numpy as np
import os,sys,glob

In [351]:
os.getcwd()
fl_wrfinput=glob.glob('wrfinput*')

In [352]:
fl_wrfinput

['wrfinput_d01.new1.nc',
 'wrfinput_d01',
 'wrfinput_d01.save.nc',
 'wrfinput_d01.nc',
 'wrfinput_d01.new',
 'wrfinput_d01.new.nc']

In [353]:
ds_disk = xr.open_dataset('wrfinput_d01.save.nc')

In [354]:
dict_rules_suews={
    'LAI_SUEWS':[('lai',1,3)],
    'albDecTr_SUEWS':0,
    'albEveTr_SUEWS':0,
    'albGrass_SUEWS':0,
    'DecidCap_SUEWS':0,
    'porosity_SUEWS':0,
    'GDD_SUEWS':[('gdd',1,5)],
    'HDD_SUEWS':[('hdd',1,6),('day',3,2)],
    'state_SUEWS':[('nsurf',1,7)],
    'soilmoist_SUEWS':[('nsurf',1,7)],
    'surf_var_SUEWS':[('nsurf',1,7)],
    'qn1_av_SUEWS':0,
    'qn1_s_SUEWS':0,
    'dqndt_SUEWS':0,
    'dqnsdt_SUEWS':0,
    'MeltWaterStore':[('nsurf',1,7)],
    'SnowAlb':0,
    'WUDay':[('wu',1,9)],
    'z0m_in':0,
    'zdm_in':0
}

In [362]:
def gen_var_expand(name,rules,var_base=ds_disk['T2'].copy(deep=True)):
    var_new=var_base.rename(name.upper());
    var_attrs=var_base.attrs
    for x_rule in rules:
#         print x_rule
        (name_dim,axis,rep) = x_rule
        var_new=np.repeat(
            var_new.expand_dims(
            name_dim,axis=axis),rep,axis=axis)
    
    var_new.attrs=var_attrs
    var_new.attrs['MemoryOrder']='XYZ'
    var_new.attrs[u'description']=name
    var_new.attrs['stagger']='Z'
    
    return var_new
    
def gen_var_keep(name,rules,var_base=ds_disk['T2'].copy(deep=True)):
    var_new=var_base.rename(name)
    var_new.attrs[u'description']=name
    
    return var_new

def gen_var(name,rules,var_base=ds_disk['T2'].copy()):
    if type(rules) is list:
        var_new=gen_var_expand(name,rules,var_base)
    else:
        var_new=gen_var_keep(name,rules,var_base)
    
    return var_new
        
            

In [363]:
ds_new=xr.Dataset({name.upper():gen_var(name,rules) for name,rules in dict_rules_suews.items()})
# ds_disk.merge(ds_new)
# ds_disk[['T2','Q2']].merge()
# ds_new[['z0m_in','zdm_in']]
ds_new['GDD_SUEWS'].attrs

OrderedDict([(u'FieldType', 104),
             (u'MemoryOrder', 'XYZ'),
             (u'description', 'GDD_SUEWS'),
             (u'units', u'K'),
             (u'stagger', 'Z')])

In [346]:
ds_disk['T2'].attrs

OrderedDict([(u'FieldType', 104),
             (u'MemoryOrder', u'XY '),
             (u'description', u'TEMP at 2 M'),
             (u'units', u'K'),
             (u'stagger', u''),
             (u'coordinates', u'XLONG XLAT XTIME')])

In [347]:
ds_disk.SMOIS.attrs

OrderedDict([(u'FieldType', 104),
             (u'MemoryOrder', u'XYZ'),
             (u'description', u'SOIL MOISTURE'),
             (u'units', u'm3 m-3'),
             (u'stagger', u'Z'),
             (u'coordinates', u'XLONG XLAT XTIME')])

In [315]:
# ds_disk.merge(ds_new)
ds_disk['SMOIS'].copy()
ds_new['SnowAlb'].attrs
ds_new['LAI_SUEWS'].attrs
for var in ds_disk.variables:
    if 'FieldType' in ds_disk[var].attrs:
        print var,ds_disk[var].ndim,ds_disk[var].attrs['FieldType']

XLAT 3 104
XLONG 3 104
LU_INDEX 3 104
ZNU 2 104
ZNW 2 104
ZS 2 104
DZS 2 104
VAR_SSO 3 104
U 4 104
V 4 104
W 4 104
PH 4 104
PHB 4 104
T 4 104
T_INIT 4 104
MU 3 104
MUB 3 104
P 4 104
PB 4 104
FNM 2 104
FNP 2 104
RDNW 2 104
RDN 2 104
DNW 2 104
DN 2 104
T_BASE 2 104
CFN 1 104
CFN1 1 104
STEP_NUMBER 1 106
THIS_IS_AN_IDEAL_RUN 1 106
P_HYD 4 104
Q2 3 104
T2 3 104
TH2 3 104
PSFC 3 104
U10 3 104
V10 3 104
RDX 1 104
RDY 1 104
DTS 1 104
DTSEPS 1 104
RESM 1 104
ZETATOP 1 104
CF1 1 104
CF2 1 104
CF3 1 104
QVAPOR 4 104
QCLOUD 4 104
QRAIN 4 104
QICE 4 104
QSNOW 4 104
QGRAUP 4 104
QNICE 4 104
QNRAIN 4 104
FCX 2 104
GCX 2 104
DTBC 1 104
TOPOSLPX 3 104
TOPOSLPY 3 104
SHDMAX 3 104
SHDMIN 3 104
SNOALB 3 104
LANDUSEF 4 104
SOILCTOP 4 104
SOILCBOT 4 104
TSLB 4 104
SMOIS 4 104
SH2O 4 104
SEAICE 3 104
IVGTYP 3 106
ISLTYP 3 106
VEGFRA 3 104
SNOW 3 104
SNOWH 3 104
CANWAT 3 104
FNDSNOWH 1 106
FNDSOILW 1 106
FNDALBSI 1 106
FNDSNOWSI 1 106
FNDICEDEPTH 1 106
LAKE_DEPTH 3 104
UOCE 3 104
VOCE 3 104
FRC_URB2D 3 104
L

In [364]:
ds_merged=ds_disk.update(ds_new)
for var in ds_merged.data_vars.keys():
    if 'coordinates' in ds_merged[var].attrs:
        print var
        del ds_merged[var].attrs['coordinates']
# ds_merged['ALBBCK']=ds_merged['ALBBCK']+0.2
ds_merged.to_netcdf('wrfinput_d01.new1.nc',mode='w',format='NETCDF4_CLASSIC')

In [164]:
rules=zip(*[('hdd',2,6),('day',4,2)])
var_base=ds_disk['T2'].copy()
var_new=var_base.copy();
var_new=var_new.expand_dims(rules[0],axis=rules[1])
var_new=np.repeat(var_new,)

In [365]:
ds_new.var()

<xarray.Dataset>
Dimensions:          ()
Data variables:
    GDD_SUEWS        float32 0.23127106
    STATE_SUEWS      float32 0.23127103
    POROSITY_SUEWS   float32 0.23127104
    ALBGRASS_SUEWS   float32 0.23127104
    DQNDT_SUEWS      float32 0.23127104
    SOILMOIST_SUEWS  float32 0.23127103
    HDD_SUEWS        float32 0.23127103
    ALBEVETR_SUEWS   float32 0.23127104
    WUDAY            float32 0.23127103
    DECIDCAP_SUEWS   float32 0.23127104
    MELTWATERSTORE   float32 0.23127103
    DQNSDT_SUEWS     float32 0.23127104
    SNOWALB          float32 0.23127104
    QN1_AV_SUEWS     float32 0.23127104
    QN1_S_SUEWS      float32 0.23127104
    SURF_VAR_SUEWS   float32 0.23127103
    ZDM_IN           float32 0.23127104
    LAI_SUEWS        float32 0.23127103
    Z0M_IN           float32 0.23127104
    ALBDECTR_SUEWS   float32 0.23127104

In [96]:
ds_disk['LAI_SUEWS'].shape
ds_disk['SMOIS'].coords['XLAT']

<xarray.DataArray 'XLAT' (Time: 1, south_north: 5, west_east: 5)>
array([[[30.308304, 30.314632, 30.316753, 30.314632, 30.308304],
        [30.849247, 30.85564 , 30.857784, 30.85564 , 30.849247],
        [31.3914  , 31.397861, 31.40002 , 31.397861, 31.3914  ],
        [31.93472 , 31.94125 , 31.943428, 31.94125 , 31.93472 ],
        [32.479176, 32.485775, 32.48798 , 32.485775, 32.479176]]],
      dtype=float32)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 30.308304 30.314632 ...
    XLONG    (Time, south_north, west_east) float32 ...
Dimensions without coordinates: Time, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LATITUDE, SOUTH IS NEGATIVE
    units:        degree_north
    stagger:      

In [146]:
xx=ds_disk['T2'].copy()
yy=np.repeat(xx.expand_dims('nsurf',axis=2),5,axis=2)
yy

<xarray.DataArray 'T2' (Time: 1, south_north: 5, nsurf: 5, west_east: 5)>
array([[[[300.1549 , 300.70944, 300.75793, 300.6841 , 301.5533 ],
         [300.1549 , 300.70944, 300.75793, 300.6841 , 301.5533 ],
         [300.1549 , 300.70944, 300.75793, 300.6841 , 301.5533 ],
         [300.1549 , 300.70944, 300.75793, 300.6841 , 301.5533 ],
         [300.1549 , 300.70944, 300.75793, 300.6841 , 301.5533 ]],

        [[299.93634, 300.7561 , 300.90884, 300.77087, 301.417  ],
         [299.93634, 300.7561 , 300.90884, 300.77087, 301.417  ],
         [299.93634, 300.7561 , 300.90884, 300.77087, 301.417  ],
         [299.93634, 300.7561 , 300.90884, 300.77087, 301.417  ],
         [299.93634, 300.7561 , 300.90884, 300.77087, 301.417  ]],

        [[300.08154, 301.05988, 301.24698, 301.01898, 301.45047],
         [300.08154, 301.05988, 301.24698, 301.01898, 301.45047],
         [300.08154, 301.05988, 301.24698, 301.01898, 301.45047],
         [300.08154, 301.05988, 301.24698, 301.01898, 301.45047]

In [251]:
ds_disk[['T2','Q2']]

<xarray.Dataset>
Dimensions:  (Time: 1, south_north: 5, west_east: 5)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 ...
    XLONG    (Time, south_north, west_east) float32 ...
Dimensions without coordinates: Time, south_north, west_east
Data variables:
    T2       (Time, south_north, west_east) float32 ...
    Q2       (Time, south_north, west_east) float32 ...
Attributes:
    TITLE:                            OUTPUT FROM REAL_EM V3.9.1.1 PREPROCESSOR
    START_DATE:                      2010-09-01_00:00:00
    SIMULATION_START_DATE:           2010-09-01_00:00:00
    WEST-EAST_GRID_DIMENSION:        6
    SOUTH-NORTH_GRID_DIMENSION:      6
    BOTTOM-TOP_GRID_DIMENSION:       30
    DX:                              60000.0
    DY:                              60000.0
    GRIDTYPE:                        C
    DIFF_OPT:                        1
    KM_OPT:                          4
    DAMP_OPT:                        0
    DAMPCOEF:                        0.2
    K

In [145]:
yy=np.repeat(yy,5,axis=2)
yy

<xarray.DataArray 'T2' (Time: 1, south_north: 5, nsurf: 25, west_east: 5)>
array([[[[300.1549, ..., 301.5533],
         ...,
         [300.1549, ..., 301.5533]],

        ...,

        [[300.4331, ..., 301.3872],
         ...,
         [300.4331, ..., 301.3872]]]], dtype=float32)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 30.308304 30.314632 ...
    XLONG    (Time, south_north, west_east) float32 120.14807 120.77399 ...
Dimensions without coordinates: Time, south_north, nsurf, west_east

In [130]:
ds_disk['LAI_SUEWS']=xx

ValueError: IndexVariable objects must be 1-dimensional

In [91]:
xx=ds_disk['T2'].copy()
yy=np.repeat(xx.expand_dims('nsurf',axis=2),5,axis=2)
yy

<xarray.DataArray 'XLAT' (Time: 1, south_north: 5, west_east: 5)>
array([[[30.308304, 30.314632, 30.316753, 30.314632, 30.308304],
        [30.849247, 30.85564 , 30.857784, 30.85564 , 30.849247],
        [31.3914  , 31.397861, 31.40002 , 31.397861, 31.3914  ],
        [31.93472 , 31.94125 , 31.943428, 31.94125 , 31.93472 ],
        [32.479176, 32.485775, 32.48798 , 32.485775, 32.479176]]],
      dtype=float32)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 30.308304 30.314632 ...
    XLONG    (Time, south_north, west_east) float32 ...
Dimensions without coordinates: Time, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LATITUDE, SOUTH IS NEGATIVE
    units:        degree_north
    stagger:      

In [129]:
xx=xr.DataArray(np.ones((1,5,7,5)))
xx

<xarray.DataArray (dim_0: 1, dim_1: 5, dim_2: 7, dim_3: 5)>
array([[[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
  

In [124]:
ds_disk.coords['XLAT']

<xarray.DataArray 'XLAT' (Time: 1, south_north: 5, west_east: 5)>
array([[[30.308304, 30.314632, 30.316753, 30.314632, 30.308304],
        [30.849247, 30.85564 , 30.857784, 30.85564 , 30.849247],
        [31.3914  , 31.397861, 31.40002 , 31.397861, 31.3914  ],
        [31.93472 , 31.94125 , 31.943428, 31.94125 , 31.93472 ],
        [32.479176, 32.485775, 32.48798 , 32.485775, 32.479176]]],
      dtype=float32)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 30.308304 30.314632 ...
    XLONG    (Time, south_north, west_east) float32 ...
Dimensions without coordinates: Time, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  LATITUDE, SOUTH IS NEGATIVE
    units:        degree_north
    stagger:      

In [63]:
for var in ds_disk.data_vars.keys():
    if 'coordinates' in ds_disk[var].attrs:
        print var
        del ds_disk[var].attrs['coordinates']

LU_INDEX
VAR_SSO
U
V
W
PH
PHB
T
T_INIT
MU
MUB
P
PB
P_HYD
Q2
T2
TH2
PSFC
U10
V10
QVAPOR
QCLOUD
QRAIN
QICE
QSNOW
QGRAUP
QNICE
QNRAIN
TOPOSLPX
TOPOSLPY
SHDMAX
SHDMIN
SNOALB
LANDUSEF
SOILCTOP
SOILCBOT
TSLB
SMOIS
SH2O
SEAICE
IVGTYP
ISLTYP
VEGFRA
SNOW
SNOWH
CANWAT
LAKE_DEPTH
UOCE
VOCE
FRC_URB2D
LAI
VAR
CON
OA1
OA2
OA3
OA4
OL1
OL2
OL3
OL4
O3RAD
MAPFAC_M
MAPFAC_U
MAPFAC_V
MAPFAC_MX
MAPFAC_MY
MAPFAC_UX
MAPFAC_UY
MAPFAC_VX
MF_VX_INV
MAPFAC_VY
F
E
SINALPHA
COSALPHA
HGT
TSK
CLAT
ALBBCK
TMN
XLAND
CPLMASK
SNOWC
SR
PCB
PC
LANDMASK
LAKEMASK
SST


In [64]:
ds_disk.to_netcdf('xx.nc')

In [21]:
da_test.name='LAI_SUEWS'

In [29]:
da_test.attrs.update(description='test var for SUEWS')
da_test

<xarray.DataArray 'LAI_SUEWS' (Time: 1, soil_layers_stag: 5, south_north: 5, west_east: 5)>
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]], dtype=float32)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 30.308304 30.314632 ...
    XLONG    (Time

In [48]:
ds_disk[['T2','Q2']].to_netcdf('xx.nc',mode='w',format='NETCDF3_CLASSIC')

In [47]:
del ds_disk['Q2'].attrs['coordinates']
del ds_disk['T2'].attrs['coordinates']

KeyError: 'coordinates'

In [50]:
ds_xx=xr.open_dataset('xx.nc')

In [51]:
ds_xx['T2']

<xarray.DataArray 'T2' (Time: 1, south_north: 5, west_east: 5)>
array([[[300.1549 , 300.70944, 300.75793, 300.6841 , 301.5533 ],
        [299.93634, 300.7561 , 300.90884, 300.77087, 301.417  ],
        [300.08154, 301.05988, 301.24698, 301.01898, 301.45047],
        [300.36032, 301.37555, 301.56662, 301.29016, 301.5196 ],
        [300.4331 , 301.2165 , 301.41998, 301.27975, 301.3872 ]]],
      dtype=float32)
Coordinates:
    XLAT     (Time, south_north, west_east) float32 ...
    XLONG    (Time, south_north, west_east) float32 ...
Dimensions without coordinates: Time, south_north, west_east
Attributes:
    FieldType:    104
    MemoryOrder:  XY 
    description:  TEMP at 2 M
    units:        K
    stagger:      